In [1]:
import ee
import datetime
import os
import itertools
import sys

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

# append the parent path into sys-path so wen can import necessary modules
sys.path.append('../')

In [3]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [4]:
from BackGround_modules.Class_1_Make_fourier_imgs import Make_Fourier
from BackGround_modules.Class_2_Classify_Fourier_Img import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

The default supplementary data a Zero img, change accordingly when classify other years imgs



### Step_0_Define_Basic_Parameters

In [5]:
# Define the time-range
start_time = '2017-01-01'
end_time   = '2019-12-31'

# Create the year_name for naming variables
year_name = f'{start_time[:4]}_{end_time[:4]}'

# Define the interval for random classifier tree_num test
Interval_index = [1] + list(range(0,121,10))[1:]



# import some spatial constrains
North_china_plain_boundary = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

# import verified points
Verified_Point   = ee.FeatureCollection(f"users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Verified_pt_{year_name}")
Verified_Point_with_city_rural = ee.FeatureCollection(f"users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Verified_pt_{year_name}_with_20K_city_rural_pts");